# Corpus Modification

In [1]:
import json
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import pandas as pd
import re
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage
from stop_words import get_stop_words
import seaborn as sns

from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import LabelEncoder

from utils import add_epoch_division, linkage_matrix, merge_corpus_poets, plot_dendrogram, remove_noise_poet, text_cleaning

In [2]:
corpus = text_cleaning(pd.read_csv("../corpora/german_poems.csv"))

In [3]:
with open("epochs.json") as f:
    epochs = json.loads(f.read())

epochs = epochs["amann"]
epoch_exceptions = ["Sturm_Drang"]
corpus = add_epoch_division(corpus, epochs, epoch_exceptions=epoch_exceptions)

In [4]:
LOWERCASE = True
MAX_FEATURES = 10000
STOP_WORDS = get_stop_words("de")

In [6]:
mod_c = merge_corpus_poets(corpus)

In [7]:
mod_c.epoch.value_counts()

Aufklärung         61
Klassik            59
Realismus          49
Barock             41
                   33
Biedermeier        28
Naturalismus       22
Expressionismus    15
Romantik           13
Name: epoch, dtype: int64

In [8]:
c = text_cleaning(mod_c)

In [9]:
c.head(5)

,id,poet,poem,year,epoch
0,0,"Abschatz, Hans Aßmann von",Was bleibt von Reichen unbekannt? Hält gleich ...,1678,Barock
1,1,"Ahlefeld, Charlotte von","Nimm hin dies Bild, das auch in weite Ferne Di...",1815,Klassik
2,2,Angelus Silesius,Der Liebe Mittelpunkt ist Gott und auch ihr Kr...,1650,Barock
3,3,"Arndt, Ernst Moritz",Wo kommst du her in dem roten Kleid Und färbst...,1814,Klassik
4,4,"Arnim, Ludwig Achim von","Fahre fort mit Liebesschlägen, Süßer Jesu, lie...",1806,Klassik
